In [1]:
import pandas as pd
import re

df2022 = pd.read_csv('data/projects2022.csv', sep=";")
df2022 = df2022.sort_values(by='votes', ascending=False).reset_index(drop=True)
df2022['ranking'] = df2022.index +1

df2022_shuffled = df2022.sample(frac=1, random_state=42).reset_index(drop=True)

df2024 = pd.read_csv('data/projects2024.csv', sep=";")
df2024 = df2024.sort_values(by='votes', ascending=False).reset_index(drop=True)
df2024['ranking'] = df2024.index +1

df2022['year'] = 2022
df2024['year'] = 2024

In [2]:
def clean_description(text):
    patron = r"^.*L'IDÉE EXPRIMÉE PAR @\S+(?:\s*:\s*)?"
    new_text = re.sub(patron, '', text)
    return new_text.strip()


df2024['description'] = df2024['description'].apply(clean_description)

In [28]:
def create_project_list_prompt(df):
    resumen_proyectos_anteriores = ""
    for i, row in df.iterrows():
        resumen_proyectos_anteriores += f"- {row['project_name']}: (Coût: {row['cost']} €, District: {row['district']}). {row['votes']} Voix, Classement: {row['ranking']} \n"
    
    return resumen_proyectos_anteriores

last_projects_results = create_project_list_prompt(df2022_shuffled)
total_budget = 400000*len(set(df2022.district.to_list()))

print(last_projects_results)
print('total budget:',total_budget)

- Vélo-rue aux Minimes: (Coût: 30000.0 €, District: 8 - Minimes / Barrière de Paris / Ponts-Jumeaux / La Vache / Raisins / Fondeyre). 40 Voix, Classement: 96 
- Des arbres pour la rue de la République (végétaliser la place Olivier): (Coût: 150000.0 €, District: 6 - Saint-Cyprien). 135 Voix, Classement: 16 
- Aménagement du site de l’observatoire de Pech-David: (Coût: 95000.0 €, District: 13 - Rangueil / Sauzelong / Jules-Julien / Pech-David / Pouvourville). 101 Voix, Classement: 31 
- Esthétique des conteneurs de recyclage: (Coût: 4000.0 €, District: 1 - Capitole / Arnaud Bernard / Carmes). 14 Voix, Classement: 159 
- Equipements sportifs: (Coût: 15000.0 €, District: 9 - Trois Cocus / Borderouge / Croix-Daurade / Paléficat / Grand Selve). 26 Voix, Classement: 129 
- Des panneaux solaires sur la Maison de Quartier de Bagatelle: (Coût: 68000.0 €, District: 16 - Fontaine-Lestang / Arènes / Bagatelle / Papus / Tabar / Bordelongue / Mermoz / La Faourette). 31 Voix, Classement: 116 
- Facill

In [35]:
with open('prompt2.txt', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

project_data = {
    'last_projects_results': last_projects_results,
    'project_name': "Réhabilitation du parc",
    'cost': 120000,
    'district': "Saint-Cyprien",
    'description': "Rénovation des aires de jeux et création d'un jardin partagé."
}

# Paso 3: Rellenar los valores en el template
prompt_filled = prompt_template.format(**project_data)

print(prompt_filled)

Vous êtes un modèle expert dans l’analyse des élections de budgets participatifs, spécifiquement dans le contexte de la ville de Toulouse, en France.

À Toulouse, la municipalité organise des élections de budget participatif pour financer des projets citoyens. Chaque habitant peut voter pour entre 1 et 3 initiatives sur le site web officiel. Les votes sont anonymes.

Voici comment les projets sont sélectionnés :
- Tous les projets sont classés selon le nombre de voix obtenues.
- Un algorithme « greedy » est utilisé : on finance les projets dans l’ordre décroissant des voix, tant que le budget total n’est pas dépassé.
- Si un projet est trop coûteux pour le budget restant, on passe au suivant.

---

Lors de l’élection de 2022 :
- Il y a eu **4532 votants**, chaque personne pouvant voter pour jusqu'à 3 projets.
- Au total, environ **11918 voix** ont été exprimées sur **200 projets**.
- Le nombre de voix par projet variait de *2* à 492**, avec une moyenne autour de **58** voix et un écart

In [ ]:
test_df['prompt'] = test_df.apply(lambda x: build_simple_election_prompt(last_projects_results,
                                                                         total_budget,
                                                                         x['project_name'],
                                                                         x['cost'],
                                                                         x['district'],
                                                                         x['description']), axis=1)

print(f'test includes {len(test_df)} projects.')

In [ ]:
import tiktoken
import numpy as np

encoding = tiktoken.encoding_for_model("gpt-4-turbo")

def tokens_counter(prompt: str, encoding) -> int:
    return len(encoding.encode(prompt))

test_df['n_tokens'] = test_df['prompt'].apply(lambda p: tokens_counter(p, encoding))

price_per_1k = 0.01  # gpt-4-turbo pricing

test_df['cost_usd'] = test_df['n_tokens'] * price_per_1k / 1000
print('experiment total cost:', np.sum(test_df.cost_usd.tolist()))

In [2]:
import requests

response = requests.post(
    "http://localhost:11434/api/generate",
    
    json={
        "model": "llama3.1:8b",
        "prompt": "que eres? hablas frances?",
        "stream": False
    }
)

print(response.json()["response"])

Soy un modelo de lenguaje artificial llamado Llama (Large Language Model Meta AI). No soy humano, pero puedo entender y responder a preguntas en varios idiomas, incluyendo francés. ¿En qué puedo ayudarte hoy? 

Puedo conversar sobre una amplia variedad de temas, desde la ciencia y la historia hasta la cultura popular y el entretenimiento. Si tienes alguna pregunta o tema específico que te gustaría discutir, no dudes en preguntarme. Hablo francés con un nivel de comprensión decente gracias a mi entrenamiento en lenguas múltiples.

¿Te gustaría conversar en francés?


In [ ]:
import requests

# Usa tu API KEY aquí
api_key = "sk-xxxxxx"

# Prompt real, ya reemplazado con tus datos:
prompt = """### First Prompt

... (todo tu texto, ya reemplazado con los datos reales)
"""

url = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

data = {
    "model": "llama3-70b-8192",  # Puedes cambiar a "mixtral-8x7b-32768" o "llama3-8b-8192"
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 512,  # ajusta según lo que necesites
    "temperature": 0.4,  # más bajo = más preciso/menos creativo
    "top_p": 1
}

response = requests.post(url, headers=headers, json=data)
print(response.json()["choices"][0]["message"]["content"])


In [ ]:
import requests

headers = {
    "Authorization": "Bearer TU_API_KEY",
    "Content-Type": "application/json"
}

data = {
    "model": "llama3-70b-8192",  # O "llama3-8b-8192" si quieres comparar
    "messages": [
        {"role": "user", "content": tu_prompt}
    ],
    "max_tokens": 512,
    "temperature": 0.4
}

response = requests.post(
    "https://api.groq.com/openai/v1/chat/completions",
    headers=headers,
    json=data
)

print(response.json()["choices"][0]["message"]["content"])
